## SVARpy: SVAR-GMM Efficiency Gains
 
This file allows to quickly recreate the SVAR and the three estimators used in the Monte Carlo simulation in Keweloh, Sascha Alexander, and Wang, Shu. "Higher Moments and  Efficiency Gains in Recursive Structural Vector Autoregressions." Oxford Bulletin of Economics and Statistics (2025).


For a full replication file see [Github](https://github.com/Saschakew/Replication-Higher-Moments-and--Efficiency-Gains-in-Recursive-Structural-Vector-Autoregressions).
 

In [ ]:
# Install required packages

# If required pip install SVARpy package
if True:
    !pip install pathos 
    !pip install SVARp 

import numpy as np 
import SVAR as SVAR
import SVAR.MoG as MoG
np.random.seed(0)

We simulate a recursive SVAR(0) $u_t=B_0 \epsilon_t$ with five variables 
\begin{align} 
\begin{bmatrix}
u_{1t} \\
u_{2t} \\
u_{3t} \\
u_{4t} \\
u_{5t} \\
\end{bmatrix} =
\begin{bmatrix}
10 & 0  & 0 & 0 & 0\\
5 & 10 & 0 & 0 & 0\\
5 & 5 & 10 & 0 & 0\\
5 & 5 & 5 & 10 & 0\\
5 & 5 & 5 & 5 & 10
\end{bmatrix}
\begin{bmatrix}
\epsilon_{1t} \\
\epsilon_{2t} \\
\epsilon_{3t} \\
\epsilon_{4t} \\
\epsilon_{5t} \\
\end{bmatrix}   
\end{align}  
 and i.i.d. structural shocks with skewness $0.9$ and excess kurtosis $2.4$ from a two-component mixture.

In [ ]:
n = 5  # Number of variables
T = 500  # Number of observations
# Specitfy B0
B0 = np.eye(n)

# shocks
mu1, sigma1 = (-0.2, np.power(0.7, 2))
mu2, sigma2 = (0.7501187648456057, np.power(1.4832737225521377, 2))
lamb = 0.7895
Omega = np.array([[mu1, sigma1], [mu2, sigma2]])

# Specitfy B_true
B_true = np.array([[1, 0  , 0  , 0, 0],
                    [0.5, 1, 0 , 0, 0],
                    [0.5, 0.5, 1, 0, 0],
                    [0.5, 0.5,  0.5  , 1, 0],
                    [0.5, 0.5,  0.5  , 0.5 , 1 ]])

V = np.linalg.cholesky(np.matmul(B_true, np.transpose(B_true)))
O = np.matmul(np.linalg.inv(V), B_true)
b_true = SVAR.SVARutil.get_Skewsym(O)
B_true = B_true * 10

# Draw structural shocks
eps = np.empty([T, n])
omega = np.zeros([n, 6])
for i in range(n):
        eps[:, i] = MoG.MoG_rnd(Omega, lamb, T)
        momentstmp = SVAR.MoG.MoG_Moments(Omega, lamb)[1:]
        for i in range(n):
            omega[i, :] = momentstmp

# Generate reduced form shocks
u = np.matmul(B_true, np.transpose(eps))
u = np.transpose(u)

 

## Estimator: Cholesky


The simulation compares three estimators, each imposing a recursive SVAR.
The first estimator $\hat{B}_{\textbf{2}}$ is the just-identified recursive SVAR  estimator based on Equation (3) obtained by applying the Cholesky decomposition.

In [ ]:
estimator = 'GMM'
estimator_name = 'Cholesky'
prepOptions = dict()
prepOptions['printOutput'] = True
prepOptions['bstartopt'] = 'Rec'
prepOptions['n_rec'] = n
prepOptions['addThirdMoments'] = False
prepOptions['addFourthMoments'] = False
prepOptions['moments_MeanIndep'] = False
prepOptions['moments_blocks'] = False
prepOptions['onlybivariate'] = False
prepOptions['Wpara'] = 'Uncorrelated'
prepOptions['Avarparametric'] = 'Uncorrelated'
prepOptions['Wstartopt'] = 'I'
prepOptions['S_func'] = False
prepOptions['kstep'] = 1 


GMM_out = SVAR.SVARest(u, estimator=estimator, prepOptions=prepOptions)


## Estimator: SVAR-CUE all moments

The second estimator $\hat{B}_{\textbf{2} + \textbf{SK}}$ is the overidentified recursive SVAR estimator based on Equation (10).

Overidentified GMM estimators with many moment conditions can suffer from a small sample bias. Therefore, the overidentified estimators are implemented using a continuous updating estimator (CUE) instead of a GMM estimator. The CUE continuously updates the weighting, that is, the weighting matrix using the assumption of independent shocks to estimate the efficient weighting matrix, see Keweloh (2023). 

In [ ]:
estimator = 'CUE'
estimator_name = 'CUE-MI'
prepOptions = dict()
prepOptions['printOutput'] = True
prepOptions['bstartopt'] = 'Rec'
prepOptions['n_rec'] = n
prepOptions['addThirdMoments'] = True
prepOptions['addFourthMoments'] = True
prepOptions['moments_MeanIndep'] = False
prepOptions['moments_blocks'] = False
prepOptions['onlybivariate'] = False
prepOptions['Wpara'] = 'Independent'
prepOptions['Avarparametric'] = 'Independent'
prepOptions['S_func'] = True
prepOptions['WupdateInOutput'] = False



GMM_out = SVAR.SVARest(u, estimator=estimator, prepOptions=prepOptions)

## Estimator: SVAR-GMM only non-redundant moments

The third estimator $\hat{B}_{\textbf{2} + \tilde{\textbf{S}} \tilde{\textbf{K}}  }$ is the overidentified bivariate recursive SVAR estimator based on Equation (15).

Overidentified GMM estimators with many moment conditions can suffer from a small sample bias. Therefore, the overidentified estimators are implemented using a continuous updating estimator (CUE) instead of a GMM estimator. The CUE continuously updates the weighting, that is, the weighting matrix using the assumption of independent shocks to estimate the efficient weighting matrix, see Keweloh (2023). 

In [ ]:
estimator = 'CUE'
estimator_name = 'CUE-MI'
prepOptions = dict()
prepOptions['printOutput'] = True
prepOptions['bstartopt'] = 'Rec'
prepOptions['n_rec'] = n
prepOptions['addThirdMoments'] = True
prepOptions['addFourthMoments'] = True
prepOptions['moments_MeanIndep'] = False
prepOptions['moments_blocks'] = False
prepOptions['onlybivariate'] = True
prepOptions['Wpara'] = 'Independent'
prepOptions['Avarparametric'] = 'Independent'
prepOptions['S_func'] = True
prepOptions['WupdateInOutput'] = False



GMM_out = SVAR.SVARest(u, estimator=estimator, prepOptions=prepOptions)

## References

Keweloh, Sascha Alexander, and Wang, Shu. "Higher Moments and  Efficiency Gains in Recursive Structural Vector Autoregressions." Oxford Bulletin of Economics and Statistics (2025) 

Keweloh, Sascha Alexander. "Structural vector autoregressions and higher moments: Challenges
and solutions in small samples." arXiv preprint arXiv:2310.08173 (2023)